In [ ]:
import json
import pathlib
import functools

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from sklearn.manifold import TSNE
from torchsummary import summary
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib import rc
from PIL import Image

%load_ext autoreload
%autoreload 2
%matplotlib inline

font = {'family': 'Times New Roman', 'weight': 'bold', 'size': 12}
rc('font', **font)

In [ ]:
seq_num = 39361

debug_file_path = f'./eval_tmp/DLA-34-FPN_box_EMM_UA_DETRAC/track_solver_debug_MVI_{seq_num:05d}.json'

with open(debug_file_path, 'rt') as in_file:
    debug_data = json.load(in_file)

In [ ]:
data_seq_dir_path = f'../../datasets/UA-DETRAC_GluonCV/raw_data/Insight-MVT_Annotation_Test/MVI_{seq_num:05d}'

frame_file_paths = list(map(str, pathlib.Path(data_seq_dir_path).iterdir()))
len(frame_file_paths)

In [ ]:
n_frames = 50

obj_ids = []
boxes = []
embs = []
frame_idxs = []

frames = debug_data['frames'][:n_frames]

for frame_idx, frame in enumerate(frames):
    input_stage = frame['stages']['input']
    embs_list = input_stage['metadata']['embeddings']
    entities = input_stage['entities']

    for entity, emb in zip(entities, embs_list):
        obj_id = entity['id']
        bbox = entity['box']
        
        obj_ids.append(obj_id)
        boxes.append(bbox)
        embs.append(emb)
        frame_idxs.append(frame_idx)

obj_ids_orig = np.asarray(obj_ids)
boxes_orig = np.asarray(boxes)
embs_orig = np.asarray(embs)
frame_idxs_orig = np.asarray(frame_idxs)

obj_ids_orig.shape, boxes_orig.shape, embs_orig.shape, frame_idxs_orig.shape

In [ ]:
subset_mask = (obj_ids_orig >= 0)
obj_ids = obj_ids_orig[subset_mask]
boxes = boxes_orig[subset_mask]
embs = embs_orig[subset_mask]
frame_idxs = frame_idxs_orig[subset_mask]
n_classes = len(np.unique(obj_ids))

obj_ids.shape, boxes.shape, embs.shape, frame_idxs.shape, n_classes

In [ ]:
def calc_reduced_embs(embs, *, n_components=2, n_iter=2000):
    tsne = TSNE(
        n_components=n_components, n_iter=n_iter, metric='cosine'
    )
    embs_reduced = tsne.fit_transform(embs)
    
    return embs_reduced

embs_reduced = calc_reduced_embs(embs)
embs_reduced.shape

In [ ]:
@functools.lru_cache(maxsize=256)
def read_image(img_file_path):
    return Image.open(img_file_path)

def read_box_crop(img_file_path, box, max_size=40):
    img = read_image(img_file_path)
    roi = img.crop(tuple(box))
    
    w_orig, h_orig = roi.size
    scale = max_size / max(w_orig, h_orig)
    w = int(round(scale * w_orig))
    h = int(round(scale * h_orig))
    roi = roi.resize((w, h))
    
    offset_img = OffsetImage(roi, zoom=1)

    return offset_img

In [ ]:
def init_colors(n_colors, *, randomize=True):
    colors =  list(mcolors.CSS4_COLORS.keys())
    if randomize:
        np.random.shuffle(colors)
    return colors[:n_colors]

class_colors = init_colors(n_classes, randomize=True)
color_seq = [
    class_colors[obj_id % len(class_colors)] for obj_id in obj_ids
]

fig, ax = plt.subplots(figsize=(12, 12))
xs = embs_reduced[:, 0]
ys = embs_reduced[:, 1]
ax.scatter(xs, ys, alpha=0.5, s=250, c=color_seq, antialiased=True)

# for x, y, obj_id, frame_idx, box in zip(xs, ys, obj_ids, frame_idxs, boxes):
#     frame_file_path = frame_file_paths[frame_idx]
#     box_roi = read_box_crop(frame_file_path, box)
#     anno_box = AnnotationBbox(box_roi, (x, y), frameon=False)
#     ax.add_artist(anno_box)

min_coord = np.min(embs_reduced)
max_coord = np.max(embs_reduced)
ax_lims = (min_coord, max_coord)

ax.set_title('Feature Embedding t-SNE Visualization')
ax.set_xlabel('X coordinate (first component)')
ax.set_ylabel('Y coordinate (second component)')
ax.set_xlim(ax_lims)
ax.set_ylim(ax_lims)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

fig.tight_layout()
fig.savefig(f'emb_tsne_vis_{n_frames}.png', dpi=400)